In [1]:
!pip install qutip
!pip install wandb
!pip install torch_harmonics
!pip install tensorly 
!pip install -U tensorly-torch

Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:

import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#import stuff from library
from neuralop.models import FNO1d
from neuralop import Trainer
from neuralop.utils import count_model_params
from neuralop import LpLoss, H1Loss
from neuralop.datasets.data_transforms import DefaultDataProcessor
from neuralop.datasets.tensor_dataset import TensorDataset
from neuralop.datasets.output_encoder import UnitGaussianNormalizer

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
def generate_random_input_states_wavefunction(N, num_states):
    input_states = torch.zeros((num_states, 2**N), dtype=torch.complex64)
    for i in range(num_states):
        # Generate random complex amplitudes for each spin state
        real_part = torch.rand(2**N) * 2 - 1  # Random real part between -1 and 1
        imag_part = torch.rand(2**N) * 2 - 1  # Random imag part between -1 and 1
        amplitudes = real_part + 1j * imag_part
        norm = torch.linalg.norm(amplitudes)  
        amplitudes /= norm  # Normalize the wavefunction
        input_states[i] = amplitudes
    return input_states

def evolve_states(input_states, hamiltonian, times):
    num_states, _ = input_states.shape
    output_states = torch.zeros_like(input_states, dtype=torch.complex64)
    for i in range(num_states):
        state = input_states[i]
        output_state = torch.zeros_like(state, dtype=torch.complex64)
        for t in times:
            evolution_operator = torch.exp(-1j * hamiltonian * t)
            evolved_state = torch.matmul(evolution_operator, state)
            output_state = evolved_state / torch.linalg.norm(evolved_state)
        output_states[i] = output_state
    return output_states


def construct_hamiltonian(n_particles, J=1):
    # Define spin operators for a single particle
    sx = (1/2) * torch.tensor([[0, 1], [1, 0]])
    sy = (1/2) * torch.tensor([[0, -1j], [1j, 0]])
    sz = (1/2) * torch.tensor([[1, 0], [0, -1]])

    hamiltonian = torch.zeros((2**n_particles, 2**n_particles), dtype=torch.complex64)
    # Create identity matrix for n_particles
    identity = torch.eye(2**(n_particles-1))

    # Extend single-particle operators to n_particles
    sx_total = torch.kron(sx, identity) + torch.kron(identity, sx)
    sy_total = torch.kron(sy, identity) + torch.kron(identity, sy)
    sz_total = torch.kron(sz, identity) + torch.kron(identity, sz)
    
    # Nearest particle interaction
    for i in range(n_particles-1):
        szsz = torch.matmul(sz_total, sz_total)
        sxsx = torch.matmul(sx_total, sx_total)
        sysy = torch.matmul(sy_total, sy_total)
        # Hamiltonian
        hamiltonian += J * (szsz + sxsx + sysy)

    return hamiltonian


def plot_state_vectors(input_states):
    N = int(np.log2(len(input_states[0])))  # Number of qubits
    for i, state in enumerate(input_states):
        fig, axs = plt.subplots(1, 2, figsize=(12, 4))  # Create two subplots side by side
        axs[0].bar(range(2**N), np.abs(state)**2, label='Real', color='b', alpha=0.7)
        axs[0].set_title(f"State {i+1} - Real Part")
        axs[0].set_xlabel("Basis State")
        axs[0].set_ylabel("Probability")
        axs[0].legend()

        axs[1].bar(range(2**N), np.imag(state)**2, label='Imaginary', color='r', alpha=0.7)
        axs[1].set_title(f"State {i+1} - Imaginary Part")
        axs[1].set_xlabel("Basis State")
        axs[1].set_ylabel("Probability")
        axs[1].legend()

        plt.tight_layout()  # Adjust layout to prevent overlapping
        plt.show()





In [4]:


"""def create_dataset(N, num_states, train_ratio, batch_size):
    # Generate input states
    input_states_wavefunction = generate_random_input_states_wavefunction(N, num_states)

    # Define evolution times
    times = torch.linspace(0, 1, 100)

    # Construct Hamiltonian
    hamiltonian = construct_hamiltonian(N)

    # Evolve input states
    output_states = evolve_states(input_states_wavefunction, hamiltonian, times)
    
    # Generate spatial grid
    spatial_grid = torch.linspace(0, 1, 2**N).unsqueeze(0).expand(num_states, -1)

    # Concatenate spatial grid with input states
    train_input = torch.cat([input_states_wavefunction.unsqueeze(-1), spatial_grid.unsqueeze(-1)], dim=-1)

    # Split data into training and testing sets
    train_size = int(train_ratio * num_states)

    train_input_final, train_output_final = train_input[:train_size], output_states[:train_size]
    test_input, test_output = train_input[train_size:], output_states[train_size:]

    print(f'[Dataset] x_train: {train_input_final.shape}, y_train: {train_output_final.shape}')
    print(f'[Dataset] x_test: {test_input.shape}, y_test: {test_output.shape}')

    # Create data loaders
    train_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(train_input_final, train_output_final),
        batch_size=batch_size,
        shuffle=True
    )
    test_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(test_input, test_output),
        batch_size=batch_size,
        shuffle=False
    )
    return train_loader, test_loader"""



"def create_dataset(N, num_states, train_ratio, batch_size):\n    # Generate input states\n    input_states_wavefunction = generate_random_input_states_wavefunction(N, num_states)\n\n    # Define evolution times\n    times = torch.linspace(0, 1, 100)\n\n    # Construct Hamiltonian\n    hamiltonian = construct_hamiltonian(N)\n\n    # Evolve input states\n    output_states = evolve_states(input_states_wavefunction, hamiltonian, times)\n    \n    # Generate spatial grid\n    spatial_grid = torch.linspace(0, 1, 2**N).unsqueeze(0).expand(num_states, -1)\n\n    # Concatenate spatial grid with input states\n    train_input = torch.cat([input_states_wavefunction.unsqueeze(-1), spatial_grid.unsqueeze(-1)], dim=-1)\n\n    # Split data into training and testing sets\n    train_size = int(train_ratio * num_states)\n\n    train_input_final, train_output_final = train_input[:train_size], output_states[:train_size]\n    test_input, test_output = train_input[train_size:], output_states[train_size:]\n\

In [5]:
def grid(spatial_dims, grid_boundary, dtype):
        """
        Parameters
        ----------
        spatial_dims : torch.size
            Sizes of spatial resolution.
        device : str
            Device where to load data ('cpu' or 'cuda:*').
        dtype : str
            Data type to encode data.

        Returns
        -------
        torch.tensor
            Output grids to concatenate.
        """
        grid_x = torch.linspace(grid_boundary[0], grid_boundary[1], spatial_dims,dtype=dtype).unsqueeze(-1)
        return grid_x


def positional_embedding_1d(data, grid_boundaries=[0, 1], batched=False):
    """Apply positional embedding as a regular 1D grid.

    Parameters
    ----------
    data : torch.tensor
        Input data to concatenate with positional embeddings.
    grid_boundaries : list, optional
        Coordinate boundaries of input grid, by default [0, 1].
    batched : bool, optional
        Whether the input data is batched, by default True.

    Returns
    -------
    torch.tensor
        Output tensor with positional embeddings concatenated.
    """


    if not batched:
        if data.ndim == 2:
            data = data.unsqueeze(-1)
    batch_size = data.shape[0]
    print(batch_size)
    print(data.shape)
    x = grid(data.shape[1],grid_boundaries, data.dtype)
    print(x.shape)
    out = torch.cat((data, x.expand(batch_size, -1, -1)),
                    dim=-1)
    print(out.shape)
    # in the unbatched case, the dataloader will stack N
    # examples with no batch dim to create one
    if not batched and batch_size == 1:
        return out.squeeze(0)
    else:
        return out


In [6]:
def data_format(x_train:torch.tensor,y_train:torch.tensor,x_test:torch.tensor,y_test:torch.tensor,
                encoding:str = "channel-wise",
                encode_input:bool = False ,encode_output:bool = False,
                grid_boundaries:list = [0,1],
                batch_size:int = 4,
                positional_encoding:bool = True
                ):

    if encode_input:
        if encoding == "channel-wise":
            reduce_dims = list(range(x_train.ndim))
        elif encoding == "pixel-wise":
            reduce_dims = [0]

        input_encoder = UnitGaussianNormalizer(dim=reduce_dims)
        input_encoder.fit(x_train)
        #x_train = input_encoder.transform(x_train)
        #x_test = input_encoder.transform(x_test.contiguous())
    else:
        input_encoder = None

    if encode_output:
        if encoding == "channel-wise":
            reduce_dims = list(range(y_train.ndim))
        elif encoding == "pixel-wise":
            reduce_dims = [0]

        output_encoder = UnitGaussianNormalizer(dim=reduce_dims)
        output_encoder.fit(y_train)
        #y_train = output_encoder.transform(y_train)
    else:
        output_encoder = None

    train_db = TensorDataset(
        x_train,
        y_train,
    )
    train_loader = torch.utils.data.DataLoader(
        train_db,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,
        pin_memory=True,
        persistent_workers=False,
    )

    test_db = TensorDataset(
        x_test,
        y_test,
    )
    test_loader = torch.utils.data.DataLoader(
        test_db,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        persistent_workers=False,
    )

    if positional_encoding:
        pos_encoding = positional_embedding_1d(x_train,grid_boundaries)
        #pos_encoding = grid(x_train.shape[1],grid_boundaries, x_train.dtype)
    else:
        pos_encoding = None
    #data_processor = DefaultDataProcessor(
        #in_normalizer=input_encoder,
        #out_normalizer=output_encoder,
       # positional_encoding=pos_encoding
    #)
    return train_loader, test_loader #, data_processor

In [7]:
def create_dataset(N,num_states,train_ratio,batch_size):
    # Generate input states
    input_states_wavefunction = generate_random_input_states_wavefunction(N, num_states)

    # Define evolution times
    times = np.linspace(0, 1, 100)

    # Construct Hamiltonian
    hamiltonian = construct_hamiltonian(N)

    # Evolve input states
    output_states = evolve_states(input_states_wavefunction, hamiltonian, times)

    input_states_wavefunction=np.array(input_states_wavefunction)
    output_states=np.array(output_states)
    # Convert to PyTorch tensors
    input_states_tensor = torch.tensor(input_states_wavefunction, dtype=torch.complex64)
    output_states_tensor = torch.tensor(output_states, dtype=torch.complex64)

    # Split data into training and testing sets
    train_size = int(train_ratio * num_states)
    train_input = input_states_tensor[:train_size]
    train_output = output_states_tensor[:train_size]
    test_input = input_states_tensor[train_size:]
    test_output = output_states_tensor[train_size:]
    
    return train_input,train_output,test_input,test_output

In [8]:
# Define parameters
N = 4  # Number of particles
num_states = 2000  # Number of input states
train_ratio = 0.8  # Ratio of training to testing data
batch_size=32
modes= (2**N)//2 #+1
hidden_channels=64

In [9]:
train_input,train_output,test_input,test_output=create_dataset(N,num_states,train_ratio,batch_size)
train_loader,test_loader= data_format(train_input,train_output,test_input,test_output, grid_boundaries = [0,2**N],batch_size=batch_size,
                positional_encoding = True)
print(train_loader)
print(test_loader)
#train_loader={'train_loader':train_loader}
#test_loader={'test_loader':test_loader}

1600
torch.Size([1600, 16, 1])
torch.Size([16, 1])
torch.Size([1600, 16, 2])


In [10]:
model = FNO1d(n_modes_height=modes,
        hidden_channels=hidden_channels,
        in_channels=2,
        out_channels=1,
        spatial_domain= 'complex')
model = model.to('cuda')

In [11]:
n_params = count_model_params(model)
print(f'\nOur model has {n_params} parameters.')


Our model has 214593 parameters.


In [12]:
optimizer = torch.optim.Adam(model.parameters(),
                                lr=8e-3,
                                weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

In [13]:
l2loss = LpLoss(d=2, p=2)
h1loss = H1Loss(d=2)

train_loss = h1loss
eval_losses={'h1': h1loss, 'l2': l2loss}

In [14]:
epochs=50
batch_size=32
device='cuda'

trainer = Trainer(model=model, n_epochs=epochs,
    device=device,
    data_processor=None,
    wandb_log=False,
    log_test_interval=1, # log at every epoch
    use_distributed=False,
    verbose=True)

trainer.train(train_loader=train_loader,
    test_loaders=test_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    regularizer=False,
    training_loss=train_loss,
    eval_losses=eval_losses,)

self.override_load_to_device=False
self.overrides_loss=False


RuntimeError: Input type (CUDAComplexFloatType) and weight type (torch.cuda.FloatTensor) should be the same